In [1]:
from lib.eugene import train, test
import torchvision.models as models

from numpy.random import RandomState
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import Subset
import torch.nn as nn
import torch.nn.functional as F

  
from torchvision import datasets, transforms

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

# We resize images to allow using imagenet pre-trained models, is there a better way?
resize = transforms.Resize(224) 

transform_val = transforms.Compose([resize, transforms.ToTensor(), normalize]) #careful to keep this one same
transform_train = transforms.Compose([resize, transforms.ToTensor(), normalize]) 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device) # you will really need gpu's for this part
    
##### Cifar Data
cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)
    
#We need two copies of this due to weird dataset api 
cifar_data_val = datasets.CIFAR10(root='.',train=True, transform=transform_val, download=True)
    
# Extract a subset of 100 (class balanced) samples per class

accs = []

for seed in [1,2]:
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 5000))
    indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:10]] for classe in range(0, 10)])
    indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[10:210]] for classe in range(0, 10)])

    train_data = Subset(cifar_data, indx_train)
    val_data = Subset(cifar_data_val, indx_val)

    print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))

    train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=128, 
                                             shuffle=True)

    val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128, 
                                           shuffle=False)


    model = models.alexnet(pretrained=True)
    model.classifier = nn.Linear(256 * 6 * 6, 10)
    optimizer = torch.optim.SGD(model.classifier.parameters(), 
                              lr=0.01, momentum=0.9,
                              weight_decay=0.0005)
    model.to(device)
    for epoch in range(10):
        train(model, device, train_loader, optimizer, epoch, display=True)

    accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over 2 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))




cuda
Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 100 Num Samples For Val 2000
Train Epoch: 0 [0/100 (0%)]	Loss: 2.437629
Train Epoch: 1 [0/100 (0%)]	Loss: 1.932911
Train Epoch: 2 [0/100 (0%)]	Loss: 0.936204
Train Epoch: 3 [0/100 (0%)]	Loss: 0.443282
Train Epoch: 4 [0/100 (0%)]	Loss: 0.137562
Train Epoch: 5 [0/100 (0%)]	Loss: 0.079731
Train Epoch: 6 [0/100 (0%)]	Loss: 0.047475
Train Epoch: 7 [0/100 (0%)]	Loss: 0.023605
Train Epoch: 8 [0/100 (0%)]	Loss: 0.011212
Train Epoch: 9 [0/100 (0%)]	Loss: 0.005826


/home/benjamin/.local/lib/python3.8/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 1.8129, Accuracy: 967/2000 (48.35%)

Num Samples For Training 100 Num Samples For Val 2000
Train Epoch: 0 [0/100 (0%)]	Loss: 2.402029
Train Epoch: 1 [0/100 (0%)]	Loss: 1.860339
Train Epoch: 2 [0/100 (0%)]	Loss: 0.988952
Train Epoch: 3 [0/100 (0%)]	Loss: 0.441356
Train Epoch: 4 [0/100 (0%)]	Loss: 0.188259
Train Epoch: 5 [0/100 (0%)]	Loss: 0.078268
Train Epoch: 6 [0/100 (0%)]	Loss: 0.038337
Train Epoch: 7 [0/100 (0%)]	Loss: 0.021586
Train Epoch: 8 [0/100 (0%)]	Loss: 0.013114
Train Epoch: 9 [0/100 (0%)]	Loss: 0.008289

Test set: Average loss: 1.5217, Accuracy: 1045/2000 (52.25%)

Acc over 2 instances: 50.30 +- 1.95
